# Multi-Body Force Field Simulation 

## Motivation

To create a model where N bodies react with each other under the influence of Newtonian Model

## Method
Used the square inverse law of gravity to find the force for each body and double integrated them using euler integration

$\vec F_i = \dfrac{Gm_im_j}{|\vec r_i - \vec r_j|^3}(\vec r_i - \vec r_j)$

$\vec V_i = \int \dfrac{\vec F_i}{m_i} dt$

$\vec P_i = \int \vec V_i dt$

and using perfectly inelastic model for collisions to determine velocities

$u_{final} = \dfrac{m_iu_i+m_ju_j}{m_i+m_j}$ 

for i in 0 to N 

and j in i to N

## What result I got

Can Simulate multiple bodies under the influence of gravity. Inelastic collision model is working. Set up a error calculation model based on energy

## Possible Future work

Use time series information of positions of N-1 bodies to find the $N^{th}$ body


Fix the error in inelastic model

## Errors

Multiple collision in a single timestep has not been possible

In [8]:
## Calling all required libraries  
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
%matplotlib tk

In [9]:
class body:
    '''
    The class creates an object representing a spherical particle with mass and radius along with initial conditions 
    '''
    def __init__(self,m,r,pos,vel):
        self.m=m
        self.r=r
        self.pos=np.array(pos)
        self.vel=np.array(vel)
        
    def collision(field):
        '''
        Add the perfectly inelastic collision mechanics and return energy residuals
        
        Input: A field object with N number of bodies under the influence of grvity
        
        '''
        energy = field.sys_energy(0)
        for i,x in enumerate(field.bodies):
            for j,y in enumerate(field.bodies[i+1:]):
                j=j+i+1
#                 print(i,j)
                if np.linalg.norm(field.pos[i]-field.pos[j])<x.r+y.r:
                    x.vel = (x.m*x.vel+y.m*y.vel)/(x.m+y.m)
                    field.vel[i]=x.vel 
                    x.m+=y.m
                    x.r= (x.r**3+y.r**3)**(1/3)
                    field.bodies= np.delete(field.bodies,j,0)
                    field.pos= np.delete(field.pos,j,0)
                    field.vel= np.delete(field.vel,j,0)
        if energy != field.sys_energy(0):
            field.E_res+= energy -  field.sys_energy(0)
            print(field.E_res)
        
class field:
    '''
    Creates an object that represents a field with various forces acting on it
    '''
    def __init__(self,bodies):
        G=1
        self.bodies= np.array(bodies)
        self.mass = (np.array([[i.m for i in self.bodies] for j in self.bodies]))
        self.pos=(np.array([i.pos for i in self.bodies]))
        self.vel=(np.array([i.vel for i in self.bodies]))
        self.E_res = 0
    def gravity_field(self): 
        
        '''
        Computes the force of gravity acting on each particle

        input: the field object

        returns: array of forces for each body in the field
        '''
        acc=np.zeros((len(self.bodies),2))
        for index,body in enumerate(self.bodies):
            magnitude= np.array([G*i.m for i in self.bodies if i!=body])
            direction=np.array([(-self.pos[index]+i)/np.linalg.norm((self.pos[index]-i))**3 for i in self.pos \
                                if (np.array_equal(self.pos[index],i))!=1])
            acc[index]=np.matmul(magnitude,direction)
            
        return acc
    
    def electric_field(self):
        
        '''
        Computes force due to charges (FUTURE POSSIBLE WORK)
        '''
        acc =np.zeros((len(self.bodies),2))
        ## add if electric field exists ##
        return acc
            
    def field_update(self,dt):
        
        '''
        Updates the state of each object in the field after a timestep using  EULER INTEGRATION

        input: The field object and the timestep
        '''
    
        acc=(self.gravity_field()+self.electric_field())
        self.pos= self.pos + self.vel*dt + acc*dt**2/2
        self.vel=self.vel + acc*dt
        for i,body in enumerate(self.bodies):
            body.pos= self.pos[i]
            body.vel= self.vel[i]
        
    def sys_momentum(self):
        '''
        Prints the total vectorial momentum of all objects in the field
        
        input: The field object
        '''
        self.mass = (np.array([[i.m for i in self.bodies] for j in self.bodies]))
        print("The total momentum is ", np.matmul(self.mass,self.vel).sum(axis=0))

    def sys_energy(self,verbose = 1):
        '''
        Computes the kinetic, potential and total energy of the system
        
        input: 
        field object
        verbose: to print the values of each type of energy
        
        returns: The energy of the system
        '''
        self.mass = (np.array([[i.m for i in self.bodies] for j in self.bodies]))
        kinetic= np.sum(np.matmul(self.mass,self.vel*self.vel))/2
        potential = 0
        for i,x in enumerate(self.bodies):
            for j,y in enumerate(self.bodies[i+1:]):
                j=j+i+1
                potential -= G*x.m*y.m/np.linalg.norm(self.pos[i]-self.pos[j])
        if verbose:
            print("Kinetic:",kinetic)
            print("Potential:",potential)
            print("Total:",kinetic + potential+self.E_res)
        
        return kinetic + potential + self.E_res
    
    def body_finder(self):
        '''
        Uses to positions of N-1 objects to find position of Nth object (FUTURE WORK)
        '''
        pass

In [10]:
# Setting up the Gravitational constant
G=10**-3.5
iterations=100000
time_step = 0.001

# Initializing the solar system or the field of particles
solar_system= field([body(100,0.01,[0,0],[0,-0.015]),body(10,0.01,[1,0],[0,0.15]),body(1,0.01,[0.5,0],[0.2,0.2]),\
             body(1,0.01,[0.5,0.5],[0.2,0]),])
solar_system.sys_energy(1)
pos=[]
## Running the simulations number of iterations
for _ in tqdm_notebook(range(iterations)):
    solar_system.field_update(time_step)
    body.collision(solar_system)
    pos.append(solar_system.pos)
    
# Computing the total energy before and after simulation to calculate the error
solar_system.sys_energy(1)
pos=np.array(pos)

Kinetic: 0.735
Potential: -0.43562382557757123
Total: 0.29937617442242875


0.727878323882563

Kinetic: 0.3257389436338295
Potential: -0.3691356346367855
Total: 0.6844816328796071


In [ ]:
pos=pos[::200]

In [31]:
fig, ax = plt.subplots()
for i in tqdm_notebook(range(len(pos))):
    ax.axis([-2, 2, -2, 2])
    x= pos[i][:,0]
    y= pos[i][:,1]
    ax.scatter(x,y,s=4,color='b')
    fig.canvas.draw()
    plt.pause(0.001)
    fig.canvas.flush_events()
    plt.cla()